In [1]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [3]:
from graph_rag.rag_store import GraphRAGStore
from graph_rag.rag_extractor import GraphRAGExtractor
from graph_rag.rag_query_engine import GraphRAGQueryEngine
from llama_index.core import PropertyGraphIndex
from llama_index.llms.openai import OpenAI
import textwrap

import data_pull_and_prep.utils as utils
from graph_rag.utils import TranscriptionParser


In [5]:
transcription_with_char_timestamps = utils.import_pkl_file(project_root+"/data/audio_1/ivanka_trump_transcription_char_timestamps.pkl")

In [7]:
kg_extractor = GraphRAGExtractor(max_paths_per_chunk=4)

In [8]:
graph_store = GraphRAGStore(username="neo4j", password="neo4j_rishi", url="bolt://localhost:7687")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [11]:
index = PropertyGraphIndex(
    nodes=TranscriptionParser.get_nodes(transcription_with_char_timestamps),
    kg_extractors=[kg_extractor],
    property_graph_store=graph_store,
    show_progress=True,
)

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.89it/s]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [17]:
index.property_graph_store.build_communities()

In [18]:
default_llm = OpenAI(model="gpt-3.5-turbo")

query_engine = GraphRAGQueryEngine(
    graph_store=index.property_graph_store,
    llm=default_llm,
    index=index,
    similarity_top_k=10,
    num_children=10,
)

In [20]:
query_str = "What are some famous quotes mentioned in this podcast and who said them?"
response = query_engine.custom_query(query_str)
print("----------------------")
print(textwrap.fill(response, width=100))

----------------------
1. "The beauty of life is in the small moments." - Marcus Aurelius 2. "The only way to do great work
is to love what you do." - Steve Jobs 3. "Balance is not something you find, it's something you
create." - Jana Kingsford 4. "Life is like riding a bicycle. To keep your balance, you must keep
moving." - Albert Einstein 5. "The happiness of your life depends upon the quality of your
thoughts." - Marcus Aurelius 6. "The only thing we have to fear is fear itself." - Franklin D.
Roosevelt 7. "In the end, it's not the years in your life that count. It's the life in your years."
- Abraham Lincoln 8. "The best way to predict the future is to create it." - Peter Drucker 9. "Those
who have a 'why' to live, can bear with almost any 'how'." - Victor Frankl


In [ ]:
entities = self.get_entities(query_str, self.similarity_top_k)
self.entities = entities
community_ids = self.retrieve_entity_communities(
    self.graph_store.entity_info, entities
)
self.community_ids = community_ids
community_summaries = self.graph_store.get_community_summaries()

In [22]:
entities = query_engine.get_entities(query_str, query_engine.similarity_top_k)

In [25]:
def retrieve_entity_communities(entity_info, entities):
    community_ids = []
    for entity in entities:
        if entity in entity_info:
            community_ids.extend(entity_info[entity])

In [27]:
query_engine.graph_store.entity_info

In [26]:
retrieve_entity_communities(
    query_engine.graph_store.entity_info, entities
)

TypeError: argument of type 'NoneType' is not iterable